Removing Criteria:
- Adult Column (Has values other than 'TRUE' and 'FALSE')
- release_date (can seprate it into three columns year, month and date) and drop month and date. (rather pull it during inner join with IMDB movies)
-budget and revenue column, finance related columns have different currencies (needs to be standardized)
- Merging Files together. 
- Currency Conversion and Removing Dollar signs.
-



In [ ]:
#For Mounting the google drive in colab notebook, to access files saved in the folder stored in drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#One line code to load the python script to work with "R" language in colab notebook.
#After loading this, each cell that needs execution in "R", should start with "%%R"
%load_ext rpy2.ipython


In [ ]:
from rpy2.robjects.vectors import StrVector
from rpy2.robjects.packages import importr
utils = importr("utils")
utils.install_packages("priceR")

In [ ]:
#To check na values column wise
%%R
fun <- function(x){
    tmp <- is.na(x)
    apply(tmp,2,sum)
}

In [ ]:
#Loading the csv file "movies_metadata.csv" from the folder in the drive, and storing it into the variable "stage_1"
%%R
library("tidyverse")
library("readr")
library("stringr")
library("dplyr")
library("priceR")
library("tibble")


stage_1 <- as_tibble(read_csv("/content/drive/MyDrive/IDMP PROJECT/movies_metadata.csv"))
colnames(stage_1)

In [ ]:
 #Dividing the list of columns into the ones that are to be kept and the ones to be reserved
%%R
to_keep_columns <- c("adult","genres", "imdb_id","popularity","runtime","vote_count","production_countries","original_language","title")
drop_columns <- c("belongs_to_collection","homepage","id","budget","poster_path","video","tagline","production_companies","overview","release_date","revenue","status","original_title","vote_average")

In [ ]:
## Function to convert columns containing Dictionaries to List:
%%R
getAttribute <- function(vector) {
    vector <- as.vector(str_split(vector, regex("[\\[{'':,}\\]]"))[[1]])
    vector <- vector[!vector == "" & !vector == " " ]
    vector <- as.vector(vector[which(vector =="name")+1])
    return(toString(vector))
}

In [ ]:
%%R
## Converts Currecy as per today's curr value:
convert_currency <- function(datum)
{  
  #retrives a list of currencies seen in datum
  curr_type = unique(str_sub(datum,1,4))
  
  for (curr in curr_type){
    #Fetches the currency Valye using priceR package
    exch_rate = exchange_rate_latest(curr)
    conversion_value = as.double(exch_rate[exch_rate[1] == "USD"])[2]
    # Retrieved values in data with curr currency
    sub_datum = datum[str_sub(datum,1,4)==curr]
    for (data in sub_datum)
    {
      ind = which(datum == data)
      value = as.double(str_sub(data,5))
      res = as.integer(value * conversion_value)
      datum[ind] = res
    }
  }
  return(datum)
}

In [ ]:
%%R
csc <- function(x,colname,df){
    
    ncols <- NULL
    colm <- NULL
    ncols <- max(stringr::str_count(x,", ")) + 1
    colm <- paste(colname,1:ncols,sep="_")
    
    df <- tidyr::separate(data = df, col = colname, sep = ", ", into = colm, remove = FALSE)
    unique_val_list <- data.frame(matrix(ncol = 1, nrow = 0))
    colnames(unique_val_list) <- colm[1]
    for(i in colm)
    { 
      colnames(unique_val_list) <- i
      tmp <- as.data.frame(unique(df[,i]))
      colnames(tmp) <- i 
      unique_val_list <- rbind(as.data.frame(unique_val_list),tmp)
    }

    unique_val_list <- as.data.frame(unique(unique_val_list))
    unique_val_list <- as.data.frame(na.omit(unique_val_list))
    
    for(i in 1:length(unique_val_list[,1]))
    {
        df[unique_val_list[i,1]] <- 0
    }

    for(i in 1:nrow(df))
    {
      for(j in colm)
      { 
        if(!is.na(df[i,j]))
        {
            k <- as.character(df[i,j])
            df[i,k] = 1
        } 
      }
    }
    df <- select(df, -colm)
    return (df) 
}

In [ ]:
## Keeping necessary Columns only.
%%R
stage_1 <- stage_1[to_keep_columns]

In [ ]:
## Converting all Dictionary kinda Cols into Lists
%%R
stage_1$genres <- sapply(stage_1$genres,getAttribute, USE.NAMES = FALSE, simplify = "array") # Genres Column
stage_1$production_countries <- sapply(stage_1$production_countries,getAttribute, USE.NAMES = FALSE, simplify = "array")



In [ ]:
#Replacing blank values with NA and then omitting the NAs.
%%R
stage_1 <- stage_1 %>%
  mutate(genres = ifelse(genres == '', NA, genres)) %>%
  mutate(production_countries = ifelse(production_countries == '', NA, production_countries))


In [ ]:
# Joining the files movies metadata and IMDB movies.
%%R
IMDB_movies <- as_tibble(read_csv("/content/drive/MyDrive/IDMP PROJECT/IMDb movies.csv"))
IMDB_rating <- as_tibble(read_csv("/content/drive/MyDrive/IDMP PROJECT/IMDb ratings.csv"))
stage_1 <- dplyr::inner_join(stage_1,
                             select(IMDB_movies,year,imdb_title_id,director,budget,worlwide_gross_income),
                             by = c("imdb_id" = "imdb_title_id"))
stage_1 <- dplyr::inner_join(stage_1,
                             select(IMDB_rating,imdb_title_id,weighted_average_vote),
                             by = c("imdb_id" = "imdb_title_id"))
stage_1 <- na.omit(stage_1)

In [ ]:
%%R
stage_1$budget[!str_detect(stage_1$budget, "^\\$")] = convert_currency(stage_1$budget[!str_detect(stage_1$budget, "^\\$")])# Currency Conversion
stage_1$worlwide_gross_income[!str_detect(stage_1$worlwide_gross_income, "^\\$")] = convert_currency(stage_1$worlwide_gross_income[!str_detect(stage_1$worlwide_gross_income, "^\\$")]) # Currency Conversion
stage_1 = na.omit(stage_1)

stage_1$budget[str_detect(stage_1$budget, "^\\$")]= as.numeric(str_sub(stage_1$budget[str_detect(stage_1$budget, "^\\$")],3)) # Dollar removal
stage_1$worlwide_gross_income[str_detect(stage_1$worlwide_gross_income, "^\\$")]= as.numeric(str_sub(stage_1$worlwide_gross_income[str_detect(stage_1$worlwide_gross_income, "^\\$")],3)) #Dollar Removal
stage_1$budget = as.numeric(stage_1$budget)
stage_1$worlwide_gross_income = as.numeric(stage_1$worlwide_gross_income)

stage_1 = stage_1 %>% 
  mutate("hit/not" = ifelse(worlwide_gross_income/ budget > 1.0, 1, 0))

stage_1 <- na.omit(stage_1)

Stage 1 data
-genre (unnecesary values that are not genre)
deleting values from adult columns that are not logical

In [ ]:
%%R
stage_1 <- as.data.frame(csc(stage_1$production_countries, "production_countries",stage_1))

In [ ]:
%%R
stage_1 <- as.data.frame(csc(stage_1$genres, "genres",stage_1))


In [ ]:
%%R
lang_codes <- as_tibble(read_csv("/content/drive/MyDrive/IDMP PROJECT/language_codes_csv.csv"))
stage_1 <- dplyr::left_join(stage_1,lang_codes,
                             by = c("original_language" = "alpha2"),
                            keep = FALSE)

In [ ]:
%%R
stage_1 <- relocate(stage_1, `hit/not`, .after = last_col())

In [ ]:
%%R
write.csv(stage_1,"/content/drive/MyDrive/IDMP PROJECT/cleaned_merged.csv", row.names = FALSE)

In [ ]:
%%R
## Percentage of hits in our dataset
length(stage_1$`hit/not`[stage_1$`hit/not`==1]) / length(stage_1$`hit/not`) #/ sum(merged$`hit/not`==0)


In [ ]:
%%R
nrow(stage_1)